## Win/Loss Rating Model Prediction

Load the model and make predictions

In [1]:
import pandas as pd
import numpy as np
import pymc3 as pm
import seaborn as sns
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm
from spcl_case import *
plt.style.use('fivethirtyeight')
%matplotlib inline

### Get a list of all CS Games

In [6]:
import cfscrape # need nodejs
import json
scraper = cfscrape.create_scraper()
games = json.loads(scraper.get("http://thunderpick.com/api/matches").content)
games = pd.DataFrame(games['data'])
games = games[games.gameId == 6].sort_values('championship')

bet_games = []
for i,v in games.iterrows():
    if((v['isTournament'] == False )& (v['canWager'] == True)):
        ratio = v['matchBet']['buckets'][0]['amount']/v['matchBet']['buckets'][1]['amount']
        odds = (ratio**-1+1, ratio+1)
        wr = (odds[1]/np.sum(odds)*100., odds[0]/np.sum(odds)*100.)
        det = json.loads(scraper.get('https://thunderpick.com/api/matches/'+str(v['id'])).content)['data']
        print('Date: %s  |  Event: %s  | (BO%s) %s vs. %s  |  (%.1f:%.1f) | Total Coins: %i' % 
              (v['startTime'][:10], v['championship'], det['bestOfMaps'], v['matchBet']['buckets'][0]['label'], 
               v['matchBet']['buckets'][1]['label'], wr[0], wr[1], v['matchBet']['amount']))
        bet_games.append({'1': v['matchBet']['buckets'][0]['label'], '2': v['matchBet']['buckets'][1]['label'], 'bo': det['bestOfMaps'], 'o1': odds[0], 'o2': odds[1], 'wr': wr[0]})
bet_games = pd.DataFrame(bet_games)

Date: 2017-10-26  |  Event: AOC CS:GO CUP  | (BO3) ALTERNATE aTTaX vs. Venatores  |  (60.0:40.0) | Total Coins: 4000
Date: 2017-10-27  |  Event: Asia Minor ELEAGUE Major  | (BO1) Flash vs. B.O.O.T-d[S]  |  (60.5:39.5) | Total Coins: 4050
Date: 2017-10-27  |  Event: Asia Minor ELEAGUE Major  | (BO1) Tyloo vs. Tainted Minds  |  (63.0:37.0) | Total Coins: 4000
Date: 2017-10-27  |  Event: CIS MINOR OPEN  | (BO1) AVANGAR vs. QUANTUM BELLATOR FIRE  |  (60.0:40.0) | Total Coins: 4000
Date: 2017-10-27  |  Event: CIS MINOR OPEN  | (BO1) Tengri vs. forZE  |  (60.0:40.0) | Total Coins: 4000
Date: 2017-10-27  |  Event: ECS Season 4 Development League  | (BO1) GODSENT vs. mousesports  |  (40.0:60.0) | Total Coins: 10000
Date: 2017-10-27  |  Event: ECS Season 4 Development League  | (BO1) OpTic vs. Liquid  |  (59.4:40.6) | Total Coins: 10100
Date: 2017-10-26  |  Event: ECS Season 4 Development League  | (BO1) fnatic vs. NiP  |  (44.1:55.9) | Total Coins: 10188
Date: 2017-10-27  |  Event: ECS Season 

## Load Ratings Model

In [13]:
TEAM_SET = 'mdleu'

teams = np.load('saved_model/'+TEAM_SET+'/teams.npy')
maps = np.load('saved_model/'+TEAM_SET+'/maps.npy')
filt = np.chararray.lower(np.load('saved_model/'+TEAM_SET+'/filter_teams.npy'))
h_teams = pd.read_csv('hltv_csv/teams.csv').set_index('ID').loc[teams]
h_teams = fix_teams(h_teams)
h_teams.Name = h_teams.Name.str.lower()
h_teams_filt = h_teams[h_teams.Name.isin(filt)]

rating_model = prep_pymc_model(len(teams), len(maps))
trace = pm.backends.text.load('saved_model/'+TEAM_SET+'/trace', model=rating_model)

## Ban/Pick Predictions

In [14]:
h_bp = pd.read_csv('hltv_csv/picksAndBans.csv').set_index('Match ID')
h_matches = pd.read_csv('hltv_csv/matchResults.csv').set_index('Match ID')
h_matches['Date'] = pd.to_datetime(h_matches['Date'])
h_matches = h_matches[h_matches['Date'] >= dt.datetime(2017,1,1)]
h_bp = h_bp.join(h_matches[['Date']], how='left')
h_bp['Date'] = pd.to_datetime(h_bp['Date'])
h_bp = h_bp[h_bp['Date'] >= dt.datetime(2017,1,1)]

In [15]:
def model_mp(train, t1, t2):
    tab = train[train['Team'].isin([t1, t2])].groupby(['Team', ' Pick Type', 'Map'])['Date'].count().unstack([' Pick Type', 'Team']).fillna(0)
    return (tab/tab.sum(axis=0)).mean(level=0,axis=1)# get average

def model_played(train, t1, t2):
    a = train[train['Team 1 ID'].isin([t1,t2])].groupby(['Team 1 ID', 'Map'])['Date'].count()
    b = train[train['Team 2 ID'].isin([t1,t2])].groupby(['Team 2 ID', 'Map'])['Date'].count()
    c = pd.DataFrame([a,b], index=['a','b']).T.fillna(0)
    c = (c['a']+c['b']).unstack(level=0).fillna(0)
    return (c/c.sum()).mean(axis=1)

def predict_map(func, data, t1, t2):
    res = func(data, t1, t2)
    return res.loc[res.index != 'Default'].sort_values(ascending=False)

# Bet Predictions

In [16]:
money = 4500.
bet_games['1'] = bet_games['1'].str.lower().replace('ex-Denial', 'Denial')
bet_games['2'] = bet_games['2'].str.lower().replace('ex-Denial', 'Denial')
matches = bet_games[bet_games['1'].isin(filt) & bet_games['2'].isin(filt)].drop_duplicates()
def sig(x):
    return 1 / (1 + np.exp(-x))
def abs_norm_interval(start,end,loc,scale):
    return (norm.cdf(end,loc,scale) - norm.cdf(start,loc,scale)) + (norm.cdf(-1*start,loc,scale) - norm.cdf(-1*end,loc,scale))

t_rating = trace['rating']
t_map_rating = trace['rating | map']
t_alpha = 0.3485
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    trace_1 = t_rating[:,t1_ind]; trace_2 = t_rating[:,t2_ind]
    mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
    diff = trace_1-trace_2
    p_wl = sig(diff)
    wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
    kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
    kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
    print('%s (%.3f) vs %s (%.3f) - P:%.2f%% - %.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
          (v['1'], mr_1, v['2'], mr_2,  wr_25*100, v['wr'], wr_75*100, kelly_pct_1*100., 
           kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))

alternate attax (0.338) vs endpoint (-0.651) - P:61.79% - 60.00% - 81.44%  -  K: 5.3% (240) - 0.1% (3)
space soldiers (2.443) vs not academy (-0.185) - P:87.90% - 82.00% - 96.37%  -  K: 8.6% (387) - -0.9% (-40)
invictus aquilas (-0.206) vs pride (0.842) - P:17.38% - 27.00% - 36.61%  -  K: 0.4% (18) - 5.8% (259)
aaa (0.804) vs seed (0.645) - P:33.58% - 43.00% - 73.01%  -  K: 2.9% (130) - 2.1% (94)
invictus aquilas (-0.206) vs ago (1.872) - P:6.81% - 22.00% - 17.54%  -  K: -1.0% (-43) - 8.2% (368)
igame.com (1.074) vs nexus (1.330) - P:32.71% - 58.00% - 55.21%  -  K: 0.8% (34) - 4.1% (184)
seed (0.645) vs space soldiers (2.443) - P:8.63% - 17.74% - 22.57%  -  K: -0.3% (-11) - 7.7% (344)
ago (1.872) vs pride (0.842) - P:63.18% - 69.19% - 82.10%  -  K: 5.2% (234) - 0.6% (28)
team5 (0.026) vs igame.com (1.074) - P:16.18% - 63.00% - 38.81%  -  K: -2.4% (-108) - 6.7% (302)
extatus (1.457) vs ago (1.872) - P:29.64% - 35.35% - 51.06%  -  K: 1.5% (69) - 3.8% (172)
team5 (0.026) vs alternate atta

In [12]:
PRINT_RD_DIFF = False
for i,v in matches.iterrows():
    t1_id = h_teams_filt[h_teams_filt.Name == v['1']].index[0]; t1_ind = np.where(teams == t1_id)[0][0];
    t2_id = h_teams_filt[h_teams_filt.Name == v['2']].index[0]; t2_ind = np.where(teams == t2_id)[0][0];
    print('---------- %s vs %s ---------------------------------' % (v['1'], v['2']))
    pred_maps = predict_map(model_played, h_matches, t1_id, t2_id)
    pred_maps = pred_maps/pred_maps.sum()
    for m,s in pred_maps.iteritems():
        m_ind = np.where(maps == m)[0][0]
        trace_1 = t_map_rating[:,m_ind,t1_ind]; trace_2 = t_map_rating[:,m_ind,t2_ind]
        mr_1 = trace_1.mean(); mr_2 = trace_2.mean();
        diff = trace_1-trace_2
        p_wl = sig(diff)
        wr_25 = np.percentile(p_wl, 25); wr_75 = np.percentile(p_wl, 75)
        kappa = 32*sig(t_alpha*diff)-16.
        kelly_pct_1 = ((v['o1']*np.percentile(p_wl, 45)-(1.-np.percentile(p_wl, 45)))/v['o1'])*0.1
        kelly_pct_2 = ((v['o2']*(1.-np.percentile(p_wl, 45))-(np.percentile(p_wl, 45)))/v['o2'])*0.1
        print('    Map: %s (%.2f)  -  %s (%.3f) vs %s (%.3f) - P:%.2f%% - %.2f%% - %.2f%%  -  K: %.1f%% (%i) - %.1f%% (%i)' % 
             (m, s*100., v['1'], mr_1, v['2'], mr_2,  wr_25*100, v['wr'], wr_75*100, kelly_pct_1*100., 
               kelly_pct_1*money, kelly_pct_2*100., kelly_pct_2*money))
        
        if(PRINT_RD_DIFF):
            p_sc = [abs_norm_interval(x[0],x[1],kappa,trace['sigma'][:,m_ind]) for x in [[1.5,3.5],[3.5,5.5],[5.5,7.5],[7.5,9.5],[9.5,16]]]
            for i,sd in enumerate(['2 - 3 Rounds', '4 - 5 rounds', '6 - 7 rounds', '8 - 9 rounds', '10 rounds or more']):
                sc_25 = np.percentile(p_sc[i], 25); sc_75 = np.percentile(p_sc[i], 75)
                print('      %s : %.2f%% - %.2f%%' % (sd, sc_25*100, sc_75*100))

---------- godsent vs mousesports ---------------------------------
    Map: Mirage (22.22)  -  godsent (2.004) vs mousesports (2.940) - P:23.70% - 40.00% - 33.18%  -  K: -0.2% (-8) - 5.6% (253)
    Map: Cobblestone (16.08)  -  godsent (1.650) vs mousesports (2.931) - P:17.30% - 40.00% - 27.00%  -  K: -1.1% (-48) - 6.7% (299)
    Map: Cache (15.41)  -  godsent (1.973) vs mousesports (2.423) - P:33.01% - 40.00% - 45.19%  -  K: 1.3% (58) - 4.0% (177)
    Map: Train (14.18)  -  godsent (0.982) vs mousesports (2.392) - P:15.42% - 40.00% - 24.94%  -  K: -1.4% (-61) - 7.0% (315)
    Map: Inferno (10.56)  -  godsent (1.286) vs mousesports (2.461) - P:17.82% - 40.00% - 30.48%  -  K: -0.9% (-38) - 6.4% (288)
    Map: Overpass (10.54)  -  godsent (2.269) vs mousesports (2.436) - P:37.69% - 40.00% - 54.14%  -  K: 2.2% (98) - 2.9% (131)
    Map: Nuke (9.09)  -  godsent (1.629) vs mousesports (2.579) - P:19.33% - 40.00% - 38.76%  -  K: -0.3% (-15) - 5.8% (261)
    Map: Dust2 (1.92)  -  godsent (2.8

---------- liquid vs optic ---------------------------------
    Map: Cobblestone (21.61)  -  liquid (3.369) vs optic (2.418) - P:68.14% - 40.00% - 75.94%  -  K: 6.0% (269) - -1.4% (-63)
    Map: Train (18.32)  -  liquid (2.928) vs optic (2.665) - P:50.78% - 40.00% - 62.02%  -  K: 3.8% (169) - 1.1% (50)
    Map: Mirage (16.65)  -  liquid (2.828) vs optic (3.139) - P:36.70% - 40.00% - 48.06%  -  K: 1.8% (79) - 3.4% (152)
    Map: Inferno (14.45)  -  liquid (2.986) vs optic (2.532) - P:54.97% - 40.00% - 67.13%  -  K: 4.4% (197) - 0.4% (18)
    Map: Nuke (9.55)  -  liquid (2.242) vs optic (2.273) - P:42.57% - 40.00% - 56.12%  -  K: 2.7% (123) - 2.3% (103)
    Map: Overpass (9.39)  -  liquid (2.265) vs optic (1.964) - P:49.44% - 40.00% - 65.21%  -  K: 3.9% (173) - 1.0% (46)
    Map: Cache (9.22)  -  liquid (1.902) vs optic (1.474) - P:52.79% - 40.00% - 67.84%  -  K: 4.3% (193) - 0.5% (23)
    Map: Dust2 (0.81)  -  liquid (1.079) vs optic (2.597) - P:9.64% - 40.00% - 31.48%  -  K: -1.7% (-7

In [ ]:
np.inner(np.ones((8,25)), np.ones((8,25))).shape

In [ ]:
np.ones((8,25)).flatten().shape

In [ ]:
np.zeros((8000, 200))

In [ ]:
plt.ylim(0,1.2)
sns.kdeplot(trace_1, shade=True, alpha=0.65, legend=True, label=v['1'])
sns.kdeplot(trace_2, shade=True, alpha=0.65, legend=True, label=v['2'])

In [ ]:
h_bp.groupby('Match ID').first().count()

In [ ]:
h_bp